# VideoMagix: Multi-Template AI Content Generator

This Colab notebook lets you generate content-rich videos using templates like Story, Tutorial, Poetry, Meme, Script, and more.

**Features:**
- Prompt to video with emotion & dialogue understanding
- Image generation per scene
- Voice cloning with ElevenLabs clone-compatible open-source models
- Final video compilation with TTS + images + subtitles
- Google Drive integration to save output


In [ ]:
# Step 1: Install Dependencies
!pip install transformers diffusers accelerate
!pip install TTS moviepy gradio
!pip install git+https://github.com/openai/whisper.git
!apt-get install ffmpeg -y

In [ ]:
# Step 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Step 3: Clone Required Repositories (Image Gen + Voice Cloning)
!git clone https://github.com/huggingface/diffusers.git
!git clone https://github.com/coqui-ai/TTS.git

In [ ]:
# Step 4: Import and Setup Modules
from TTS.api import TTS
import torch, os
from moviepy.editor import *
from transformers import pipeline

# Load TTS model (you can replace with another open-source voice cloning model)
tts = TTS(model_name="tts_models/multilingual/multi-dataset/your_tts")

## Step 5: Define Your Content Prompt
Choose from the following templates: `story`, `script`, `tutorial`, `poetry`, `meme`, `video2video`, `image2video`, `clonevideo`

In [ ]:
# Example prompt
prompt = "A fantasy story about a dragon and a brave girl who saves a village."
template = "story"

In [ ]:
# Step 6: Generate Text Scenes (Simple Split for Demo)
scenes = prompt.split('.')
scenes = [s.strip() for s in scenes if s.strip()]

In [ ]:
# Step 7: Generate Images per Scene using Stable Diffusion
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

image_paths = []
for idx, scene in enumerate(scenes):
    image = pipe(scene).images[0]
    path = f"scene_{idx}.png"
    image.save(path)
    image_paths.append(path)

In [ ]:
# Step 8: Generate Voice Audio for Each Scene
audio_paths = []
for idx, scene in enumerate(scenes):
    audio_path = f"scene_{idx}.wav"
    tts.tts_to_file(text=scene, file_path=audio_path)
    audio_paths.append(audio_path)

In [ ]:
# Step 9: Combine Into Video
clips = []
for img, audio in zip(image_paths, audio_paths):
    clip = ImageClip(img).set_duration(AudioFileClip(audio).duration)
    clip = clip.set_audio(AudioFileClip(audio))
    clips.append(clip)

final_video = concatenate_videoclips(clips)
output_path = "/content/drive/MyDrive/videomagix_output.mp4"
final_video.write_videofile(output_path, fps=24)